**Define a Class and all the Relevant Functions**

In [ ]:
## Install Relevant Packages
# !pip install rdkit
# !pip install pubchempy
# !pip install bioservices
# !pip install biopython

## Import Relevant Libraries
import pandas as pd
import numpy as np
from rdkit import Chem
from rdkit.Chem import Descriptors
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from lightgbm import LGBMRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler
from bioservices import UniProt
from Bio.SeqUtils.ProtParam import ProteinAnalysis
import joblib
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam

class DrugDiscoveryPipeline:
    def __init__(self, dataset_path, external_test_path):
        self.dataset_path = dataset_path
        self.external_test_path = external_test_path
        self.data = None
        self.features = None
        self.target = None
        self.models = []
        self.best_model_info = None
        self.scaler = StandardScaler()

    ## Import Data
    def load_dataset(self):
        self.data = pd.read_csv(self.dataset_path)
        print("Dataset Summary:")
        print(self.data.info())

    ## Sample the Dataset
    def preprocess_data(self, sample_size=1000):
        sampled_df = self.data.sample(n=sample_size, random_state=42).reset_index(drop=True)
        sampled_df.dropna(inplace=True)
        self.data = sampled_df

    ## Compute Molecular Features of the Drug Molecules from SMILES Strings
    ## https://www.rdkit.org/docs/source/rdkit.Chem.Descriptors.html
    def compute_molecular_descriptors(self, pubchem_cid):
        try:
            smiles = self.get_smiles_from_pubchem(pubchem_cid)
            if smiles:
                mol = Chem.MolFromSmiles(smiles)
                if mol:
                    return [
                        Descriptors.MolWt(mol),
                        Descriptors.MolLogP(mol),
                        Descriptors.NumHDonors(mol),
                        Descriptors.NumHAcceptors(mol),
                        Descriptors.TPSA(mol),
                        Descriptors.FractionCSP3(mol),
                        Descriptors.NumRotatableBonds(mol),
                        Descriptors.NumAromaticRings(mol),
                        Descriptors.HeavyAtomCount(mol)
                    ]
        except Exception:
            pass
        return [np.nan] * 9

    ## Compute Protein Features relavant to Drug-Protein Binding from corresponding Amino Acid Sequences
    ## https://biopython.org/docs/1.75/api/Bio.SeqUtils.ProtParam.html
    def encode_protein_features(self, uniprot_id):
        try:
            sequence = self.get_protein_sequence(uniprot_id)
            print(f"Protein sequence for {uniprot_id}: {sequence}")

            if sequence:
                analysis = ProteinAnalysis(sequence)
                features = {
                    "length": len(sequence),
                    "aromaticity": analysis.aromaticity(),
                    "instability_index": analysis.instability_index(),
                    "isoelectric_point": analysis.isoelectric_point(),
                    "gravy": analysis.gravy(),
                    "molecular_weight": analysis.molecular_weight(),
                    "flexibility_mean": np.mean(analysis.flexibility()),
                    "extinction_coeff_reduced": analysis.molar_extinction_coefficient()[0],
                    "extinction_coeff_disulfide": analysis.molar_extinction_coefficient()[1],
                }
                secondary_structures = analysis.secondary_structure_fraction()
                features.update({
                    "helix_fraction": secondary_structures[0],
                    "sheet_fraction": secondary_structures[1],
                    "coil_fraction": secondary_structures[2],
                })
                aa_composition = analysis.get_amino_acids_percent()
                features.update(aa_composition)
                return list(features.values())
        except Exception as e:
            print(f"Error analyzing sequence for UniProt ID {uniprot_id}: {e}")
            return [np.nan] * len(features)  # Return NaNs in case of failure
        return [np.nan] * len(features)

    ## Retrieve SMILES Strings from the Pubchem IDs
    ## https://pubchempy.readthedocs.io/en/latest/api.html
    def get_smiles_from_pubchem(self, pubchem_cid):
        import pubchempy as pcp
        try:
            compound = pcp.Compound.from_cid(pubchem_cid)
            return compound.isomeric_smiles
        except Exception:
            return None

    ## Retrieve Amino Acid Sequence of the Proteins from the UniProt IDs
    ## https://bioservices.readthedocs.io/en/latest/_modules/bioservices/uniprot.html
    def get_protein_sequence(self, uniprot_id):
        uniprot_service = UniProt()
        try:
            result = uniprot_service.retrieve(uniprot_id, frmt="fasta")
            sequence = ''.join(result.split('\n')[1:])
            return sequence
        except Exception:
            return None

    ## Create Dataframes from Drug Features and Protein Features and then Combine them
    def extract_features(self):
        compound_features = self.data['pubchem_cid'].astype(int).apply(self.compute_molecular_descriptors)
        compound_features_df = pd.DataFrame(compound_features.tolist(), columns=[
            'MolWt', 'MolLogP', 'NumHDonors', 'NumHAcceptors', 'TPSA',
            'FractionCSP3', 'NumRotatableBonds', 'NumAromaticRings', 'HeavyAtomCount'
        ])

        print(compound_features_df)

        protein_features = self.data['UniProt_ID'].apply(self.encode_protein_features)
        protein_features_df = pd.DataFrame(protein_features.tolist(), columns=[
            "length", "aromaticity", "instability_index", "isoelectric_point", "gravy",
            "molecular_weight", "flexibility_mean", "extinction_coeff_reduced",
            "extinction_coeff_disulfide", "helix_fraction", "sheet_fraction", "coil_fraction"
        ] + [f"aa_{aa}" for aa in "ACDEFGHIKLMNPQRSTVWY"])

        # Convert kiba_score_estimated column to numeric using map()
        self.data['kiba_score_estimated'] = self.data['kiba_score_estimated'].map({True: 1, False: 0})

        # Combine all features into a single DataFrame
        self.features = pd.concat([
            compound_features_df, protein_features_df,
            self.data[['kiba_score_estimated']].reset_index(drop=True)
        ], axis=1)

        # Drop NaN rows and separate target
        self.features.dropna(inplace=True)
        self.target = self.data['kiba_score']

    ## Train-Test Split
    def train_test_split(self):
        return train_test_split(self.features, self.target, test_size=0.2, random_state=42)

    ## Model Evaluation
    def train_and_evaluate(self, model, X_train, y_train, X_test, y_test):
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        rmse = np.sqrt(mean_squared_error(y_test, y_pred))
        r2 = r2_score(y_test, y_pred)
        return {"RMSE": rmse, "R2": r2, "model": model}

    ## Run Different Machine Learning Models and Save the Best Model
    def run_ml_models(self, X_train, y_train, X_test, y_test):
        self.models = [
            {"name": "Linear Regression", "model": LinearRegression()},
            {"name": "LightGBM", "model": LGBMRegressor(n_estimators=1000, learning_rate=0.05, random_state=42, verbose=-1)},
            {"name": "Random Forest", "model": RandomForestRegressor(n_estimators=100, random_state=42)}
        ]

        self.best_model_info = {"name": None, "rmse": float('inf'), "model": None}

        for model_info in self.models:
            metrics = self.train_and_evaluate(model_info["model"], X_train, y_train, X_test, y_test)
            print(f"{model_info['name']} - RMSE: {metrics['RMSE']:.4f}, R2: {metrics['R2']:.4f}\n")

            if metrics["RMSE"] < self.best_model_info["rmse"]:
                self.best_model_info = {
                    "name": model_info["name"],
                    "rmse": metrics["RMSE"],
                    "model": metrics["model"]
                }

        # Save the best ML model
        joblib.dump(self.best_model_info["model"], f"best_ml_model_{self.best_model_info['name'].replace(' ', '_')}.pkl")
        print(f"Best ML model ({self.best_model_info['name']}) saved as best_ml_model_{self.best_model_info['name'].replace(' ', '_')}.pkl")

    ## Build a Deep Neural Network for Model Training and Save it
    def build_and_train_dnn(self, X_train_scaled, y_train, X_test_scaled, y_test):
        dnn_model = Sequential([
            Dense(128, activation='relu', input_shape=(X_train_scaled.shape[1],)),
            Dropout(0.2),
            Dense(64, activation='relu'),
            Dropout(0.2),
            Dense(32, activation='relu'),
            Dense(1)
        ])

        dnn_model.compile(optimizer=Adam(learning_rate=0.001), loss='mse', metrics=['mae'])
        dnn_model.fit(X_train_scaled, y_train, validation_split=0.2, epochs=50, batch_size=32, verbose=1)

        # Save the trained model
        dnn_model.save('dnn_model.h5')  # Saves the model to a file
        # dnn_model.save('/content/dnn_model.h5')
        print("DNN model saved successfully as dnn_model.h5")

        y_pred_dnn = dnn_model.predict(X_test_scaled).flatten()
        rmse_dnn = np.sqrt(mean_squared_error(y_test, y_pred_dnn))
        r2_dnn = r2_score(y_test, y_pred_dnn)

        return dnn_model, {"RMSE": rmse_dnn, "R2": r2_dnn}

    ## Load External Dataset for Testing the best ML model and the Deep Learning Model
    def load_external_test_data(self):
        external_test_data = pd.read_csv(self.external_test_path)
        external_test_data.dropna(inplace=True)
        return external_test_data

    ## Predict Binding Affinity Between Drug-Protein Pairs in the External Test Dataset and Save the results
    def predict_external(self, external_test_data):
        external_test_data['pubchem_cid'] = external_test_data['pubchem_cid'].astype(int)
        compound_features = external_test_data['pubchem_cid'].apply(self.compute_molecular_descriptors)
        compound_features_df = pd.DataFrame(compound_features.tolist(), columns=[
            'MolWt', 'MolLogP', 'NumHDonors', 'NumHAcceptors', 'TPSA',
            'FractionCSP3', 'NumRotatableBonds', 'NumAromaticRings', 'HeavyAtomCount'
        ])

        protein_features = external_test_data['UniProt_ID'].apply(self.encode_protein_features)
        protein_features_df = pd.DataFrame(protein_features.tolist(), columns=[
            "length", "aromaticity", "instability_index", "isoelectric_point", "gravy",
            "molecular_weight", "flexibility_mean", "extinction_coeff_reduced",
            "extinction_coeff_disulfide", "helix_fraction", "sheet_fraction", "coil_fraction"
        ] + [f"aa_{aa}" for aa in "ACDEFGHIKLMNPQRSTVWY"])

        external_features = pd.concat([
            compound_features_df, protein_features_df,
            external_test_data[['kiba_score_estimated']].reset_index(drop=True)
        ], axis=1)

        external_features.dropna(inplace=True)
        external_features_scaled = self.scaler.transform(external_features)

        ml_predictions = self.best_model_info["model"].predict(external_features)
        dnn_model = load_model('dnn_model.h5')
        dnn_predictions = dnn_model.predict(external_features_scaled).flatten()

        external_test_data['kiba_score_pred_ml'] = ml_predictions
        external_test_data['kiba_score_pred_dnn'] = dnn_predictions
        external_test_data.to_csv('external_test_predictions.csv', index=False)

        return external_test_data


**Define a Pipeline and Run the Pipeline Step by Step for Model Training**



In [ ]:
pipeline = DrugDiscoveryPipeline(dataset_path='Deloitte_DrugDiscovery_dataset.csv', external_test_path='external_test_dataset.csv')

pipeline.load_dataset()

pipeline.preprocess_data()

pipeline.extract_features()

pipeline.train_test_split()

pipeline.run_ml_models(X_train, y_train, X_test, y_test)

pipeline.build_and_train_dnn(X_train_scaled, y_train, X_test_scaled, y_test)


**Run the Pipeline for testing the Best ML model and the Deep Neural Network Model with External Dataset**





In [ ]:
pipeline.load_external_test_data()

pipeline.predict_external(external_test_data)